In [1]:
import pandas as pd
import numpy as np

In [2]:
congresults08 = pd.read_csv("/Users/yiliwang/2008congresults.csv")

### Data cleaning

In [3]:
genele= congresults08[pd.notnull(congresults08['GENERAL..'])] # extract general election candidate

In [4]:
genele = genele.loc[genele['DISTRICT'] != "S"] # get rid of district "S" 

##### Find Explanatory Notes here: http://www.fec.gov/pubrec/fe2008/federalelections2008.shtml

In [5]:
genele['DIS'] = pd.Series(genele['STATE.ABBREVIATION'] + genele['DISTRICT'], index=genele.index)

In [6]:
genele['GENERAL..'].replace(regex=True,inplace=True,to_replace=r'%',value=r'') 
# remove % from GENERAL.. for the convinence of calculation
genele['GENERAL.'].replace(regex=True,inplace=True,to_replace=r',',value=r'') 
genele = genele[genele["GENERAL."]!= "Unopposed"] # drop GERNAL. == unopposed rows (GENERAL..==n/a)
genele = genele[genele["GENERAL.."]!= 'n/a']

In [7]:
genele_es = genele[['STATE','STATE.ABBREVIATION','DIS','CANDIDATE.NAME','PARTY','GENERAL.','GENERAL..']] # extract useful columns
genele_es.index = range(1,len(genele_es) + 1) # reindex

##### This is the dataset that we are used to conduct the further calculation. It contains general election candidate name, party, vote, and percentage of vote (each district candidates' percentage of vote sum up to 1). It contains 56 states/ regions total (includes unincorporated U.S. territory like Puerto Rico)

In [8]:
genele_es

,STATE,STATE.ABBREVIATION,DIS,CANDIDATE.NAME,PARTY,GENERAL.,GENERAL..
1,Alabama,AL,AL01,"Bonner, Jo",R,210660,98.27
2,Alabama,AL,AL01,Scattered,W,3707,1.73
3,Alabama,AL,AL02,"Bright, Bobby",D,144368,50.23
4,Alabama,AL,AL02,"Love, Jay",R,142578,49.61
5,Alabama,AL,AL02,Scattered,W,448,0.16
6,Alabama,AL,AL03,"Rogers, Mike",R,142708,54.03
7,Alabama,AL,AL03,"Segall, Joshua",D,121080,45.84
8,Alabama,AL,AL03,Scattered,W,332,0.13
9,Alabama,AL,AL04,"Aderholt, Robert",R,196741,74.76
10,Alabama,AL,AL04,"Sparks, Nicholas B.",D,66077,25.11


In [9]:
dict_of_dis = {k: v for k, v in genele_es.groupby('DIS')} # create a dictionary to store each district
dict_of_dis['CA01']["GENERAL.."] # for example, search for a certain district

63    68.10
64    23.36
65     8.54
66     0.00
Name: GENERAL.., dtype: object

##### Nation ratio for house representative party: D/R

In [10]:
nation_rep_party = [None]*len(genele_es["DIS"].unique())
state = [None]*len(genele_es["DIS"].unique())
state_ab = [None]*len(genele_es["DIS"].unique())

In [11]:
for i in range(len(genele_es["DIS"].unique())):
    cd = genele_es["DIS"].unique()[i]
    nation_rep_party[i] = genele_es["PARTY"][genele_es.loc[genele_es['DIS']==cd, 'GENERAL..'].idxmax()]
    state[i] = genele_es["STATE"][genele_es.loc[genele_es['DIS']==cd, 'GENERAL..'].idxmax()]
    state_ab[i] = genele_es["STATE.ABBREVIATION"][genele_es.loc[genele_es['DIS']==cd, 'GENERAL..'].idxmax()]
    #print genele_es["PARTY"][genele_es.loc[genele_es['DIS']==cd, 'GENERAL..'].idxmax()]

In [12]:
from collections import Counter
Counter(nation_rep_party) # whole country: D/R = 1.329

Counter({'CRV': 1,
         'D': 238,
         'D/R': 1,
         'DFL': 5,
         'DNL': 1,
         'D\xa0': 1,
         'GRE': 2,
         'GRE   ': 1,
         'I': 2,
         'LBT': 1,
         'N': 1,
         'NPP': 1,
         'R': 179,
         'R\xa0': 3,
         'W': 3,
         'WF': 3})

In [13]:
us_rep_party = pd.DataFrame({'State': state,'STATE.ABBREVIATION': state_ab,'Winner Party': nation_rep_party})
us_rep_party

,STATE.ABBREVIATION,State,Winner Party
0,AL,Alabama,R
1,AL,Alabama,D
2,AL,Alabama,R
3,AL,Alabama,R
4,AL,Alabama,D
5,AL,Alabama,R
6,AL,Alabama,D
7,AK,Alaska,R
8,AS,American Samoa,D
9,AZ,Arizona,D


#####  State ratio for house representative party: D/R

In [14]:
from __future__ import division

In [15]:
StateRatio = [None]*len(us_rep_party['State'].unique())

In [16]:
dict_of_state = {k: v for k, v in genele_es.groupby('STATE')} # create a dictionary to store each district
#dict_of_state['ALABAMA']["GENERAL.."] 

In [17]:
def State_Ratio_Generator(state):
    count_each_state = Counter(us_rep_party[us_rep_party['State']==state]['Winner Party'])
    if count_each_state['R']!=0:
        StateRatio_D_R = count_each_state['D']/count_each_state['R']
    else:
        StateRatio_D_R = "D" # all D when denominator = count_each_state['R'] = 0
    return StateRatio_D_R

In [18]:
# For example: search for a state ratio
State_Ratio_Generator('American Samoa')

'D'

In [19]:
for i in range(len(us_rep_party['State'].unique())):
    StateRatio[i] = State_Ratio_Generator(us_rep_party['State'].unique()[i])

In [20]:
us_rep_party['State'].unique()

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbi', 'Florida', 'Georgia', 'Guam', 'Hawaii',
       'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Northern Mariana Islands',
       'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virgin Islands', 'Virginia',
       'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [21]:
state_rep_party = pd.DataFrame({'State': us_rep_party['State'].unique(),'State Abbreviation':us_rep_party['STATE.ABBREVIATION'].unique(),'State D/R Ratio': StateRatio})

In [22]:
state_rep_party # DataFrame of Each State's D/R ratio

,State,State Abbreviation,State D/R Ratio
0,Alabama,AL,0.75
1,Alaska,AK,0
2,American Samoa,AS,D
3,Arizona,AZ,1.66667
4,Arkansas,AR,2
5,California,CA,1.55
6,Colorado,CO,2.5
7,Connecticut,CT,D
8,Delaware,DE,0
9,District of Columbi,DC,D


### Precinct-level Voting Data (Rdata)

In [23]:
# In R, we are missing the following regions 2008 precinct-level election data:
# American Samoa(AS), Arkansas(AR), Florida(FL),Guam(GU), Illinois(IL), Indiana(IN),
# Marshall Islands(MH), Micronesia(FM), Nevada(NV), Northern Marianas(MP), Palau(PW),
# Puerto Rico(PR), Virgin Islands(VI), West Virginia(WV)

In [24]:
ratio_result_precinct_vote = pd.read_csv("/Users/yiliwang/ratio_result_precinct_vote.csv")

In [25]:
del ratio_result_precinct_vote ['Unnamed: 0']

In [27]:
combined_party_ratio = pd.merge(ratio_result_precinct_vote,state_rep_party,left_on='state_name',right_on='State Abbreviation',how='inner')

In [28]:
del combined_party_ratio['State Abbreviation']

In [101]:
combined_party_ratio = combined_party_ratio[np.isfinite(combined_party_ratio['vote_ratio'])] # delete row with NAN

In [126]:
list(combined_party_ratio['State D/R Ratio'])

[0.0,
 0.75,
 2.0,
 1.55,
 2.5,
 'D',
 0.0,
 'D',
 1.5,
 1.0,
 'D',
 0.0,
 'D',
 7.0,
 'D',
 1.1428571428571428,
 0.0,
 0.8,
 1.5,
 0.0,
 1.6,
 'D',
 0.0,
 'D',
 1.6,
 'D',
 8.0,
 1.25,
 0.25,
 4.0,
 1.7142857142857142,
 'D',
 0.5,
 'D',
 1.25,
 0.6,
 0.8333333333333334,
 2.0,
 1.6666666666666667,
 0.0]

##### Calculate the "supposed" D/R Representative Ratio based on actual voting data

In [110]:
list_state = combined_party_ratio['state_name']

In [106]:
num_cd = [None]*combined_party_ratio.shape[0] # store the number of US House of Representative in this particular state

In [115]:
for i in range(len(list_state)):
    num_cd[i] = len(genele_es[genele_es['STATE.ABBREVIATION'] == list_state.iloc[i]]['DIS'].unique())

Voting ratio for each state: D/R

In [156]:
vote_rto = [float(item) for item in genele_es['GENERAL..']] # convert str into float
genele_es['vote_rto'] = vote_rto

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [222]:
combined_party_ratio['num_cd'] = num_cd

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [230]:
def find_party_actual_num(statename, party):
    df_vote_ratio = genele_es[genele_es['STATE.ABBREVIATION'] == statename]
    no = sum([float(i) for i in df_vote_ratio[df_vote_ratio['PARTY'] == party]['GENERAL.']])
    tot = sum([float(i) for i in df_vote_ratio['GENERAL.']])
    
    cd_no = float(combined_party_ratio[combined_party_ratio['state_name'] == statename]['num_cd'])
    
    actual_num = (no/tot)*cd_no
    return actual_num

In [231]:
actual_USR_R = [find_party_actual_num(party, 'R') for party in list_state]
actual_USR_D = [find_party_actual_num(party, 'D') for party in list_state]

In [245]:
actual_USR_D

[0.44974730107452254,
 2.7104272805869556,
 1.5834020373656776,
 31.733234708201433,
 3.861051844385842,
 2.9748644591229927,
 0.37989710914576724,
 1.5315091508289236,
 2.5602680753658564,
 0.8145337789957545,
 0.0,
 1.5834754023648165,
 8.620651533867616,
 5.371561983576946,
 1.2164551239894044,
 7.847024023580817,
 0.0,
 4.507253629650213,
 1.9376413689009988,
 0.3242462050322312,
 7.07496204710074,
 0.0,
 1.0248349879674696,
 1.0808311418941443,
 7.229172653709445,
 1.6836020653943327,
 17.507615347493065,
 9.729587074878031,
 1.8834760611461956,
 3.0792435582811146,
 11.082751251708602,
 1.3858869274722065,
 2.944235787586251,
 0.6755574435300667,
 4.674376869391825,
 12.663769810730304,
 5.478545359646177,
 5.327857653365302,
 3.9883221736727137,
 0.9357376628891213]

In [232]:
actual_voting_USR_ratio = [None]*len(actual_USR_R)

In [233]:
for i in range(len(actual_USR_R)):
    if actual_USR_R[i] == 0:
        actual_voting_USR_ratio[i] = 'D'
    else: 
        actual_voting_USR_ratio[i] = actual_USR_D[i]/actual_USR_R[i]

In [234]:
RESULT = pd.DataFrame({'0State Abbreviation':list_state,'Real Support Ratio': actual_voting_USR_ratio, 'Elected D/R Ratio':list(combined_party_ratio['State D/R Ratio'])})

In [235]:
# Set D = 10 for the convience of plotting
RESULT = RESULT.replace(['D'],[10]) 

In [246]:
len(actual_USR_D)

40

In [247]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=RESULT['0State Abbreviation'],
    y=RESULT['Elected D/R Ratio'],
    name='Elected D/R Ratio'
)
trace2 = go.Bar(
    x=RESULT['0State Abbreviation'],
    y=RESULT['Real Support Ratio'],
    name='Real Support Ratio'
)

data = [trace1, trace2]
layout = go.Layout(
    title='2008 Democratic v.s Republican Election Result and Support Ratio Difference',
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

## Plot

In [80]:
states = list(genele_es['STATE.ABBREVIATION'].unique())

In [81]:
def find_sprt_ratio(state):
    
    state_df = genele_es[genele_es['STATE.ABBREVIATION']==state]
    state_d_list = [int(i) for i in state_df[state_df['PARTY'] == 'D']['GENERAL.']]
    state_r_list  = [int(i) for i in state_df[state_df['PARTY'] == 'R']['GENERAL.']]
    
    den = sum(state_r_list)+sum(state_d_list)
    
    if den == 0:
        ratio_support = 0
    else:
        ratio_support = (sum(state_r_list)-sum(state_d_list))/(sum(state_r_list)+sum(state_d_list))
        
    return ratio_support

In [82]:
sprt_ratio = [find_sprt_ratio(state) for state in states] # store support ratio

In [83]:
df0 = pd.DataFrame(np.column_stack([states, sprt_ratio]),columns=['state', 'support ratio'])

In [ ]:
df_plot = pd.merge(df0,df,left_on='state',right_on='code',how='inner')

In [57]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.plotly as py
import plotly 
import plotly.graph_objs as go

In [93]:
plotly.tools.set_credentials_file(username='YiliWang', api_key='GLJL915ia4PtDL3BNtIg')

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')

scl = [[-1.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

data = [ dict(
        type='choropleth',
        colorscale = 'RdBu',
        autocolorscale = False,
        locations = df_plot['code'],
        z = df_plot['support ratio'],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Party Support Ratio")
        ) ]

layout = dict(
        title = '2008 US Republic v.s Democratic Party Support Ratio (state-level)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )